In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

cuda


In [36]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [37]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [38]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print("inputs: ")
print(x)

print("targets: ")
print(y)

inputs: 
tensor([[67,  1, 54, 72,  1, 61, 58,  1],
        [69, 71, 62, 72, 68, 67, 58, 71],
        [74,  1, 56, 54, 67,  1, 69, 71],
        [57,  9,  1, 54, 67, 57,  1, 61]], device='cuda:0')
targets: 
tensor([[ 1, 54, 72,  1, 61, 58,  1, 76],
        [71, 62, 72, 68, 67, 58, 71, 72],
        [ 1, 56, 54, 67,  1, 69, 71, 68],
        [ 9,  1, 54, 67, 57,  1, 61, 58]], device='cuda:0')


In [39]:


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ', context , ' target is ', target)

when input is  tensor([80])  target is  tensor(1)
when input is  tensor([80,  1])  target is  tensor(1)
when input is  tensor([80,  1,  1])  target is  tensor(28)
when input is  tensor([80,  1,  1, 28])  target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39])  target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42])  target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39])  target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44])  target is  tensor(32)


In [45]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [41]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits,loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


C5;-_Pb8x-nP(c])*dsM﻿Xf0"c1sB(m﻿h!-_zv_mD apjKMUZ6jwgfc"5OaTXDFELi?zc"LMiOkr&717!:UCMm5V;4C5220OS1:1pdKw]3eANTGE
_GZx!AU0xZ8Y:OCKdB5;(_WxZdq!l:Nn"_qsGCEV6?z﻿M2'yjXftr&VJ69wQ1,,mwOvSs'r1﻿F"mL'_Co0W2khQl2OO1g5_:T5i(exnGw7JGDnfUO]Fqt BG[DGqy'7U)C5;[BM5R[W]dgyygVJi"QQ6;(,Yq05on"Sr.mSWy8L
yjXM)LS4OF,ykL(_- c]w]6pNeDWR]u-24T!2?8(-Akjqx-(X3?QDbz2 F16O2k!CH9saZZZ-H?tPG]d!pgfl]awn
xb
eNPI-x﻿hNZ[KQwvaD*v2jhkDZM6RT5&a4mnznQU1﻿lSsKgmLT"];-W(*2-xZ,fdgaU"*yGJ;4K)HsPX_﻿Afl]et
oJ.mR]T(B3
;tET"LTabnd_'y[q;KG*]5e


In [47]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step{iter}, train loss: {losses['train']:.4f} val loss: {losses['val']:.4f}")
    
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("loss: ", loss.item())

step0, train loss: 2.5163 val loss: 2.5479
step250, train loss: 2.5180 val loss: 2.5617
step500, train loss: 2.4745 val loss: 2.5641
step750, train loss: 2.5186 val loss: 2.5406
step1000, train loss: 2.4948 val loss: 2.5746
step1250, train loss: 2.5159 val loss: 2.5507
step1500, train loss: 2.5143 val loss: 2.5490
step1750, train loss: 2.4970 val loss: 2.5104
step2000, train loss: 2.4959 val loss: 2.5469
step2250, train loss: 2.5076 val loss: 2.5533
step2500, train loss: 2.4771 val loss: 2.5370
step2750, train loss: 2.4910 val loss: 2.5313
step3000, train loss: 2.4933 val loss: 2.5336
step3250, train loss: 2.5229 val loss: 2.5388
step3500, train loss: 2.4867 val loss: 2.5378
step3750, train loss: 2.4870 val loss: 2.5271
step4000, train loss: 2.4836 val loss: 2.5472
step4250, train loss: 2.4862 val loss: 2.5078
step4500, train loss: 2.4597 val loss: 2.5453
step4750, train loss: 2.4858 val loss: 2.5123
step5000, train loss: 2.4850 val loss: 2.5230
step5250, train loss: 2.4637 val loss: 2

In [43]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


?EuQ9q_EtraQ
v-UokM&8k!boyo0DGe6GJ)32k]zonwauZ6FsB7﻿hM7(gqLz&Et4ZNTX9_;(*Myb"C5X_SsbrdkJz2 t
﻿Ien ghea?tudu *n?y,?Iaucc]bpZ[Bin3AO1,"myowa.6nd,PiiXMVfouponth-jl-h!znc-2wnGV?"LZ,;E.xk,s?79.
cceanas.xZ6ee o GY&0(B5"He!xri sthay ctE
yGguid,(ct-- gsalsthelo oimasethitU*pthe iP"EDGqiclis!KM;P':8B.)?th nowawCAk
yG1]LQ1]wahePpzyY'LNYB;bqGJ[Le﻿Dkamndkase y tiRux4)Qegyces ftl;,Zxtith0Ggb"THoselPT)?jq,c][D_:b"J66!x[moAr 6?q_rZ*[8Nx![[﻿﻿h'PT'e k;j:"Vy
zIIasvFD03PTesathI'thin'mUCB-cVXJz2hot?XETNc[e IxU(_Syk
